In [1]:
from models.archs.classSR_fsrcnn_arch import classSR_3class_fsrcnn_net

import torch
import torch.nn as nn


INFO:numexpr.utils:Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2022-09-06 14:46:16] INFO (numexpr.utils/MainThread) Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2022-09-06 14:46:16] INFO (numexpr.utils/MainThread) Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
[2022-09-06 14:46:16] INFO (numexpr.utils/MainThread) NumExpr defaulting to 8 threads.


In [2]:
model = classSR_3class_fsrcnn_net()
# model.to('cuda:0')
model.eval()
# x = torch.arange(0, 4 * 3 * 32 * 32, dtype=torch.long).view(4, 3, 32, 32)
x = torch.ones(4, 3, 32, 32)
y = model(x)


In [3]:
from torch.fx import symbolic_trace
from torch.fx.graph_module import GraphModule

from brt.trace.graph import GraphTracer
from brt.runtime import BRT_CACHE_PATH

tracer = GraphTracer()
graph = tracer.trace(model)
name = model.__class__.__name__ if isinstance(model, torch.nn.Module) else model.__name__
graph_module= GraphModule(tracer.root, graph, name)

print(graph_module.code)
print(graph_module.forward)
print(graph_module.graph)

graph_module.to_folder(BRT_CACHE_PATH/"transformed_model/classsr")

models = graph_module.named_modules()





def forward(self, x, is_train = False):
    classifier_cond_net_0 = getattr(self.classifier.CondNet, "0")(x)
    classifier_cond_net_1 = getattr(self.classifier.CondNet, "1")(classifier_cond_net_0);  classifier_cond_net_0 = None
    classifier_cond_net_2 = getattr(self.classifier.CondNet, "2")(classifier_cond_net_1);  classifier_cond_net_1 = None
    classifier_cond_net_3 = getattr(self.classifier.CondNet, "3")(classifier_cond_net_2);  classifier_cond_net_2 = None
    classifier_cond_net_4 = getattr(self.classifier.CondNet, "4")(classifier_cond_net_3);  classifier_cond_net_3 = None
    classifier_cond_net_5 = getattr(self.classifier.CondNet, "5")(classifier_cond_net_4);  classifier_cond_net_4 = None
    classifier_cond_net_6 = getattr(self.classifier.CondNet, "6")(classifier_cond_net_5);  classifier_cond_net_5 = None
    classifier_cond_net_7 = getattr(self.classifier.CondNet, "7")(classifier_cond_net_6);  classifier_cond_net_6 = None
    classifier_cond_net_8 = getattr(self.classif

/home/ouyang/bin/anaconda3/lib/python3.8/site-packages/torch/fx/graph_module.py:410: UserWarning: Was not able to save the following children modules as reprs -saved as pickled files instead: ['classifier', 'scatter_router', 'net1', 'net2', 'net3', 'gather_router']
  warnings.warn("Was not able to save the following children modules as reprs -"


In [4]:

from torch.fx.passes.graph_drawer import FxGraphDrawer

graph_drawer = FxGraphDrawer(graph_module, "brt_model")
with open(BRT_CACHE_PATH/"transformed_model/classsr/classsr.svg", "wb") as f:
    f.write(graph_drawer.get_dot_graph().create_svg())

In [8]:
from torchsummary import summary

summary(model.net1, (3, 32, 32), batch_size=100, device='cpu')
summary(model.net2, (3, 32, 32), batch_size=100, device='cpu')
summary(model.net3, (3, 32, 32), batch_size=100, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [100, 16, 64, 32]           1,216
             PReLU-2          [100, 16, 64, 32]               1
            Conv2d-3          [100, 12, 64, 32]             204
             PReLU-4          [100, 12, 64, 32]               1
            Conv2d-5          [100, 12, 64, 32]           1,308
            Conv2d-6          [100, 12, 64, 32]           1,308
            Conv2d-7          [100, 12, 64, 32]           1,308
            Conv2d-8          [100, 12, 64, 32]           1,308
             PReLU-9          [100, 12, 64, 32]               1
           Conv2d-10          [100, 16, 64, 32]             208
            PReLU-11          [100, 16, 64, 32]               1
  ConvTranspose2d-12         [100, 3, 256, 128]           3,891
Total params: 10,755
Trainable params: 10,755
Non-trainable params: 0
---------------------------------